<a href="https://www.kaggle.com/code/mastersniffer/transformers-agents-and-tools?scriptVersionId=227440346" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!pip install --upgrade --quiet transformers datasets evaluate soundfile librosa  


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 45.5 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.2 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.7/260.7 kB 11.7 MB/s eta 0:00:00


# Agents and tools

What is an agent?
Large Language Models (LLMs) trained to perform causal language modeling can tackle a wide range of tasks, but they often struggle with basic tasks like logic, calculation, and search. When prompted in domains in which they do not perform well, they often fail to generate the answer we expect them to.

One approach to overcome this weakness is to create an agent.

An agent is a system that uses an LLM as its engine, and it has access to functions called tools.

These tools are functions for performing a task, and they contain all necessary description for the agent to properly use them.

The agent can be programmed to:

devise a series of actions/tools and run them all at once, like the CodeAgent
plan and execute actions/tools one by one and wait for the outcome of each action before launching the next one, like the ReactJsonAgent
Types of agents
Code agent
This agent has a planning step, then generates python code to execute all its actions at once. It natively handles different input and output types for its tools, thus it is the recommended choice for multimodal tasks.


# How can I build an agent?
To initialize an agent, you need these arguments:

an LLM to power your agent - the agent is not exactly the LLM, it’s more like the agent is a program that uses an LLM as its engine.
a system prompt: what the LLM engine will be prompted with to generate its output
a toolbox from which the agent pick tools to execute
a parser to extract from the LLM output which tools are to call and with which arguments
Upon initialization of the agent system, the tool attributes are used to generate a tool description, then baked into the agent’s system_prompt to let it know which tools it can use and why.

In [2]:
!pip install transformers[agents] -qU

Build your LLM engine by defining a llm_engine method which accepts a list of messages and returns text. This callable also needs to accept a stop argument that indicates when to stop generating.

In [3]:
from huggingface_hub import login, InferenceClient

login("...")

client = InferenceClient(model="meta-llama/Meta-Llama-3-70B-Instruct")

def llm_engine(messages, stop_sequences=["Task"]) -> str:
    response = client.chat_completion(messages, stop=stop_sequences, max_tokens=1000)
    answer = response.choices[0].message.content
    return answer

You could use any llm_engine method as long as:

it follows the messages format (List[Dict[str, str]]) for its input messages, and it returns a str.
it stops generating outputs at the sequences passed in the argument stop_sequences
Additionally, llm_engine can also take a grammar argument. In the case where you specify a grammar upon agent initialization, this argument will be passed to the calls to llm_engine, with the grammar that you defined upon initialization, to allow constrained generation in order to force properly-formatted agent outputs.

You will also need a tools argument which accepts a list of Tools - it can be an empty list. You can also add the default toolbox on top of your tools list by defining the optional argument add_base_tools=True.

Now you can create an agent, like CodeAgent, and run it. You can also create a TransformersEngine with a pre-initialized pipeline to run inference on your local machine using transformers. For convenience, since agentic behaviours generally require stronger models such as Llama-3.1-70B-Instruct that are harder to run locally for now, we also provide the HfApiEngine class that initializes a huggingface_hub.InferenceClient under the hood.

In [4]:
from transformers import CodeAgent, HfApiEngine

llm_engine = HfApiEngine(model="meta-llama/Meta-Llama-3-70B-Instruct")
agent = CodeAgent(tools=[], llm_engine=llm_engine, add_base_tools=True)

agent.run(
    "Could you translate this sentence from French, say it out loud and return the audio.",
    sentence="Où est la boulangerie la plus proche?",
)

Failed to load tokenizer for model meta-llama/Meta-Llama-3-70B-Instruct: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Meta-Llama-3-70B-Instruct.
403 Client Error. (Request ID: Root=1-67d30ff4-7b1d302b6194877049cd5fc8;340d352a-18ea-4b7f-be8c-20b2526923a3)

Cannot access gated repo for url https://huggingface.co/meta-llama/Meta-Llama-3-70B-Instruct/resolve/main/config.json.
Access to model meta-llama/Meta-Llama-3-70B-Instruct is restricted and you are not in the authorized list. Visit https://huggingface.co/meta-llama/Meta-Llama-3-70B-Instruct to ask for access.. Loading default tokenizer instead.


tokenizer_config.json:   0%|          | 0.00/3.76k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/801k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

======== New task ========
Could you translate this sentence from French, say it out loud and return the audio.
You have been provided with these initial arguments: {'sentence': 'Où est la boulangerie la plus proche?'}.


BadRequestError: (Request ID: Root=1-67d30ffa-4312c5675486b64652bc983d;e0b0c80f-43c5-44e0-86c8-736be012d319)

Bad request:
Model requires a Pro subscription; check out hf.co/pricing to learn more. Make sure to include your HF token in your query.

In [5]:
from transformers import CodeAgent

agent = CodeAgent(tools=[], add_base_tools=True)

agent.run(
    "Could you translate this sentence from French, say it out loud and give me the audio.",
    sentence="Où est la boulangerie la plus proche?",
)

Failed to load tokenizer for model meta-llama/Meta-Llama-3.1-8B-Instruct: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Meta-Llama-3.1-8B-Instruct.
403 Client Error. (Request ID: Root=1-67d31044-2d6e35a26f1893134f93e9bc;9d83f376-2a27-448f-9ac5-9b5f8a95caca)

Cannot access gated repo for url https://huggingface.co/meta-llama/Meta-Llama-3.1-8B-Instruct/resolve/main/config.json.
Access to model meta-llama/Llama-3.1-8B-Instruct is restricted and you are not in the authorized list. Visit https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct to ask for access.. Loading default tokenizer instead.
======== New task ========
Could you translate this sentence from French, say it out loud and give me the audio.
You have been provided with these initial arguments: {'sentence': 'Où est la boulangerie la plus proche?'}.


BadRequestError: (Request ID: Root=1-67d31045-5340732628ca2be2505d696c;f44227df-3475-4110-831f-b7f769fd718a)

Bad request:
Model requires a Pro subscription; check out hf.co/pricing to learn more. Make sure to include your HF token in your query.

Note that we used an additional sentence argument: you can pass text as additional arguments to the model.

You can also use this to indicate the path to local or remote files for the model to use:

In [6]:
from transformers import ReactCodeAgent

agent = ReactCodeAgent(tools=[], llm_engine=llm_engine, add_base_tools=True)

agent.run("Why does Mike not know many people in New York?", audio="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/transformers/recording.mp3")

======== New task ========
Why does Mike not know many people in New York?
You have been provided with these initial arguments: {'audio': 'https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/transformers/recording.mp3'}.
Error in generating llm output: (Request ID: Root=1-67d31051-6fddebd20807e12f7ea2d600;66da31f2-86c4-438c-9858-ffd5027452d8)

Bad request:
Model requires a Pro subscription; check out hf.co/pricing to learn more. Make sure to include your HF token in your query..
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_http.py", line 409, in hf_raise_for_status
    response.raise_for_status()
  File "/usr/local/lib/python3.10/dist-packages/requests/models.py", line 1024, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPError: 400 Client Error: Bad Request for url: https://router.huggingface.co/hf-inference/models/meta-llama/Meta-Llama-3-70B-Instruct/v

'Error in generating final llm output: (Request ID: Root=1-67d31052-2975509007723dab52acd22a;7bf1bad1-488e-4588-90aa-99143166878e)\n\nBad request:\nModel requires a Pro subscription; check out hf.co/pricing to learn more. Make sure to include your HF token in your query..'

In [7]:
# The prompt and output parser were automatically defined, 
# but you can easily inspect them by calling the system_prompt_template 
# on your agent.

print(agent.system_prompt_template)

You are an expert assistant who can solve any task using code blobs. You will be given a task to solve as best you can.
To do so, you have been given access to a list of tools: these tools are basically Python functions which you can call with code.
To solve the task, you must plan forward to proceed in a series of steps, in a cycle of 'Thought:', 'Code:', and 'Observation:' sequences.

At each step, in the 'Thought:' sequence, you should first explain your reasoning towards solving the task and the tools that you want to use.
Then in the 'Code:' sequence, you should write the code in simple Python. The code sequence must end with '<end_action>' sequence.
During each intermediate step, you can use 'print()' to save whatever important information you will then need.
These print outputs will then appear in the 'Observation:' field, which will be available as input for the next step.
In the end you have to return a final answer using the `final_answer` tool.

Here are a few examples using

It’s important to explain as clearly as possible the task you want to perform. Every run() operation is independent, and since an agent is powered by an LLM, minor variations in your prompt might yield completely different results. You can also run an agent consecutively for different tasks: each time the attributes agent.task and agent.logs will be re-initialized.

# Code execution
A Python interpreter executes the code on a set of inputs passed along with your tools. This should be safe because the only functions that can be called are the tools you provided (especially if it’s only tools by Hugging Face) and the print function, so you’re already limited in what can be executed.

The Python interpreter also doesn’t allow imports by default outside of a safe list, so all the most obvious attacks shouldn’t be an issue. You can still authorize additional imports by passing the authorized modules as a list of strings in argument additional_authorized_imports upon initialization of your ReactCodeAgent or CodeAgent:

In [8]:
from transformers import ReactCodeAgent

agent = ReactCodeAgent(tools=[], additional_authorized_imports=['requests', 'bs4'])
agent.run("Could you get me the title of the page at url 'https://huggingface.co/blog'?")


Failed to load tokenizer for model meta-llama/Meta-Llama-3.1-8B-Instruct: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Meta-Llama-3.1-8B-Instruct.
403 Client Error. (Request ID: Root=1-67d310e7-0c6723736bd8f98c67fd67f3;34a90d34-1d6a-4d07-a909-86f896d211f0)

Cannot access gated repo for url https://huggingface.co/meta-llama/Meta-Llama-3.1-8B-Instruct/resolve/main/config.json.
Access to model meta-llama/Llama-3.1-8B-Instruct is restricted and you are not in the authorized list. Visit https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct to ask for access.. Loading default tokenizer instead.
======== New task ========
Could you get me the title of the page at url 'https://huggingface.co/blog'?
Error in generating llm output: (Request ID: Root=1-67d310e8-632352053fd86d2e7ada8ba5;f0551b63-1e55-4146-9835-215cfb1f0acb)

Bad request:
Model requires a Pro subscription; check out hf.co/pricing to learn more. Make sure to include you

'Error in generating final llm output: (Request ID: Root=1-67d310e9-12cf0ddb605199751c32a4b2;b6b94449-9868-46b6-9afc-5d2ace688125)\n\nBad request:\nModel requires a Pro subscription; check out hf.co/pricing to learn more. Make sure to include your HF token in your query..'

The execution will stop at any code trying to perform an illegal operation or if there is a regular Python error with the code generated by the agent.

# Tools
A tool is an atomic function to be used by an agent.

You can for instance check the PythonInterpreterTool: it has a name, a description, input descriptions, an output type, and a __call__ method to perform the action.

When the agent is initialized, the tool attributes are used to generate a tool description which is baked into the agent’s system prompt. This lets the agent know which tools it can use and why.

# Default toolbox
Transformers comes with a default toolbox for empowering agents, that you can add to your agent upon initialization with argument add_base_tools = True:

Document question answering: given a document (such as a PDF) in image format, answer a question on this document (Donut)
Image question answering: given an image, answer a question on this image (VILT)

Speech to text: given an audio recording of a person talking, transcribe the speech into text (Whisper)

Text to speech: convert text to speech (SpeechT5)

Translation: translates a given sentence from source language to target language.

DuckDuckGo search*: performs a web search using DuckDuckGo browser.

Python code interpreter: runs your the LLM generated Python code in a secure environment. This tool will only be added to ReactJsonAgent if you initialize it with add_base_tools=True, since code-based agent can already natively execute Python code

You can manually use a tool by calling the load_tool() function and a task to perform.

In [ ]:
from transformers import load_tool

tool = load_tool("text-to-speech")
audio = tool("This is a text to speech tool")

# Create a new tool
You can create your own tool for use cases not covered by the default tools from Hugging Face. For example, let’s create a tool that returns the most downloaded model for a given task from the Hub.

You’ll start with the code below.

In [10]:
from huggingface_hub import list_models

task = "text-classification"

model = next(iter(list_models(filter=task, sort="downloads", direction=-1)))
print(model.id)

cross-encoder/ms-marco-MiniLM-L6-v2


This code can quickly be converted into a tool, just by wrapping it in a function and adding the tool decorator:

In [11]:
from transformers import tool

@tool
def model_download_tool(task: str) -> str:
    """
    This is a tool that returns the most downloaded model of a given task on the Hugging Face Hub.
    It returns the name of the checkpoint.

    Args:
        task: The task for which
    """
    model = next(iter(list_models(filter="text-classification", sort="downloads", direction=-1)))
    return model.id

The function needs:

1) A clear name. The name usually describes what the tool does. Since the code returns the model with the most downloads for a task, let’s put model_download_tool.
2) Type hints on both inputs and output
3) A description, that includes an ‘Args:’ part where each argument is described (without a type indication this time, it will be pulled from the type hint). All these will be automatically baked into the agent’s system prompt upon initialization: so strive to make them as clear as possible!

In [ ]:
from transformers import CodeAgent
agent = CodeAgent(tools=[model_download_tool], llm_engine=llm_engine)
agent.run(
    "Can you give me the name of the model that has the most downloads in the 'text-to-video' task on the Hugging Face Hub?"
)